In [0]:
# STEP 1 — Imports
from pyspark.sql.types import *
from pyspark.sql.functions import *


# Load tables directly from catalog (default schema)
customers = spark.table("workspace.default.customers_75")
hotels    = spark.table("workspace.default.hotels_75")
bookings  = spark.table("workspace.default.bookings_75")

# Quick sanity check
print("Customers:", customers.count())
print("Hotels:", hotels.count())
print("Bookings:", bookings.count())


# Join bookings with customers and hotels
joined_df = (
    bookings
    .join(customers, "customer_id")
    .join(hotels, "hotel_id")
)

# Preview joined data
joined_df.select(
    "booking_id",
    "customer_id",
    "customer_name",
    "hotel_id",
    "hotel_name",
    "booking_date",
    "amount"
).show(10, truncate=False)

from pyspark.sql.functions import col, avg

# Convert stay_nights column to integer and then calculate average
avg_duration = (
    bookings
    .withColumn("stay_nights_int", col("stay_nights").cast("int"))  # cast to int
    .groupBy("hotel_id")
    .agg(avg("stay_nights_int").alias("avg_stay_nights"))
    .join(hotels, "hotel_id")
    .select("hotel_name", "avg_stay_nights")
    .orderBy(col("avg_stay_nights").desc())
)

display(avg_duration)


from pyspark.sql.functions import month, col, count

# Seasonal booking trends by month
seasonal_trends = (
    bookings
    .withColumn("month", month("booking_date"))   # extract month from date
    .groupBy("month")
    .agg(count("*").alias("total_bookings"))
    .orderBy("month")
)

# Show results as Databricks interactive table
display(seasonal_trends)

from pyspark.sql.functions import sum, col

# Hotels with highest revenue
hotel_revenue = (
    bookings
    .withColumn("amount_num", col("amount").cast("double"))   # ensure amount is numeric
    .groupBy("hotel_id")
    .agg(sum("amount_num").alias("total_revenue"))
    .join(hotels, "hotel_id")                                # join to get hotel names
    .select("hotel_name", "total_revenue")
    .orderBy(col("total_revenue").desc())                    # sort by revenue
)

# Show results as interactive table
display(hotel_revenue)

from pyspark.sql.functions import col, avg

# Average stay duration by city
avg_stay_city = (
    bookings
    .withColumn("stay_nights_int", col("stay_nights").cast("int"))   # ensure stay_nights is numeric
    .groupBy("hotel_id")
    .agg(avg("stay_nights_int").alias("avg_stay_duration"))
    .join(hotels, "hotel_id")                              # bring city info
    .groupBy("city")
    .agg(avg("avg_stay_duration").alias("avg_stay_by_city"))
    .orderBy(col("avg_stay_by_city").desc())
)

# Show results as Databricks interactive table
display(avg_stay_city)


from pyspark.sql.functions import col, lower, trim, count

# Hotels with highest cancellations (case-insensitive check)
cancellations = (
    bookings
    .withColumn("status_clean", lower(trim(col("status"))))   # clean status column
    .filter(col("status_clean") == "cancelled")               # filter cancelled
    .groupBy("hotel_id")
    .agg(count("*").alias("num_cancellations"))               # count cancellations
    .join(hotels, "hotel_id")                                 # bring hotel names
    .select("hotel_name", "num_cancellations")
    .orderBy(col("num_cancellations").desc())
)

# Display results as interactive table
display(cancellations)


Customers: 75
Hotels: 75
Bookings: 75
+----------+-----------+--------------+--------+-----------------+------------+--------+
|booking_id|customer_id|customer_name |hotel_id|hotel_name       |booking_date|amount  |
+----------+-----------+--------------+--------+-----------------+------------+--------+
|1         |36         |Ishaan Reddy  |72      |Garden Suites 72 |2024-01-11  |1875.0  |
|2         |63         |Anika Sharma  |72      |Garden Suites 72 |2024-09-03  |6000.0  |
|3         |12         |Ishaan Iyer   |38      |Lake Retreat 38  |2024-08-14  |15092.77|
|4         |55         |Saanvi Das    |71      |Lake Retreat 71  |2024-12-04  |13861.97|
|5         |25         |Vihaan Joshi  |41      |Fort Residency 41|2024-05-02  |59810.38|
|6         |68         |Arjun Rao     |37      |Garden Palace 37 |2024-04-12  |47398.56|
|7         |69         |Aaradhya Joshi|17      |Sky Resort 17    |2024-10-07  |12606.7 |
|8         |14         |Rohan Reddy   |2       |Sea Residency 2  |2025-0

hotel_name,avg_stay_nights
Grand Palace 29,7.0
Garden Palace 37,7.0
Metro Retreat 8,7.0
City Suites 10,7.0
City Lodge 43,7.0
Grand Retreat 65,7.0
Mountain Inn 6,6.5
Fort Residency 41,6.0
Mountain Resort 59,6.0
Sea Palace 62,6.0


Databricks visualization. Run in Databricks to view.

month,total_bookings
1,8
2,10
3,7
4,7
5,4
6,4
7,7
8,9
9,7
10,4


Databricks visualization. Run in Databricks to view.

...

Databricks visualization. Run in Databricks to view.

city,avg_stay_by_city
Mumbai,5.833333333333334
Chennai,4.805555555555555
Goa,4.75
Delhi,4.5
Kochi,4.5
Ahmedabad,4.333333333333334
Jaipur,4.333333333333333
Pune,4.0
Kolkata,3.5555555555555554
Bengaluru,3.375


Databricks visualization. Run in Databricks to view.

hotel_name,num_cancellations
Garden Suites 72,2
Grand Haven 54,2
Sky Resort 17,1
Grand Lodge 67,1
City Suites 10,1
Garden Hotel 35,1
City Palace 26,1
Grand Palace 29,1
Fort Residency 34,1
Sea Inn 3,1


Databricks visualization. Run in Databricks to view.